 # <b><span style="color: #ff4c4c">  Network Anomaly Detection </span>

## Goals & Objectives:


Network parameters data, with 4000 samples, 35 features. Need to build an anomaly detection prediction model.

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
%matplotlib inline
import sys

print('python:',sys.version)
print('Numpy:',np.__version__)
print('Pandas:',pd.__version__)
print('Scikitlearn:',sklearn.__version__)
print('Seaborn: ',sns.__version__)
print('matplotlib:',matplotlib.__version__)


In [ ]:
NA=pd.read_csv(r'Network Anomaly Detection Dataset_all.csv')
NA


# I. EDA 

### 1) Data Summarization:

In [ ]:
NA.shape
NA.info()


In [ ]:
NA.describe()


In [ ]:
NA.isnull().any()


In [ ]:
from collections import Counter

NA['class'].info()
Counter(NA['class'])


In [ ]:
numeric=list(NA.describe().columns)
categoric=list(set(NA.columns).difference(numeric))
NA[categoric]


In [ ]:
plt.figure(figsize=(20,11))
sns.boxplot(NA[numeric])
plt.xticks(rotation=45)
plt.show()

### 2) Data Transformation : 

#### Z score Data transformation

In [ ]:
from scipy import stats

z_scores = stats.zscore(NA[numeric])
outliers = NA[(z_scores < -2) | (z_scores > 2)]
outliers.describe()
#z_scores.describe()
outliers.isna()#.sum()
co=[]
for a in outliers.columns:
    co.append(outliers[a].isna().sum()) #Counter(
co


In [ ]:
plt.figure(figsize=(20,11))
sns.boxplot(z_scores)
plt.xticks(rotation=90)
plt.show()


In [ ]:
Q1 = NA[numeric].quantile(0.25)
Q3 = NA[numeric].quantile(0.75)
IQR = Q3 - Q1
outliers = NA[NA[numeric] < (Q1 - 1.5 * IQR) | (NA[numeric] > (Q3 + 1.5 * IQR))]
outliers.describe()
#Counter(outliers['ifoutDiscards11'])


In [ ]:
plt.figure(figsize=(15,28))
#mask for upper triangle
mask=np.zeros_like(z_scores[numeric].corr(), dtype=bool)
mask[np.triu_indices_from(mask)]=True
mask = np.logical_and(mask, (z_scores[numeric].corr() > -0.5) | (z_scores[numeric].corr() < 0.5))
#Generate custom diverging colormap
#cmap=sns.color_palette("flare")
cmap=sns.diverging_palette(h_neg=12, h_pos=12, s=100, l=40, sep=2, n=10, center='light', as_cmap=False)
#Heatmap with mask with correct aspect ratio
sns.heatmap(z_scores[numeric].corr(), mask=mask, cmap=cmap, vmin=-1,vmax=1,
            center=0, square=True, linewidths=0.5,annot=True, fmt='.2f', cbar_kws={'shrink':0.5})
plt.show()


#### Ln transform

In [ ]:
zero_count = NA[numeric].eq(0).sum()
zero_count


In [ ]:
for a in numeric:
    for j in range(0,len(NA[numeric])):
        if NA[a][j]==0:
            NA[a][j]+=0.0001
zero_count


In [ ]:
for column in NA.columns:
    NA.loc[NA[column] == 0, column] += 0.0001
NA[numeric].eq(0).sum()


In [ ]:
NA2=np.log(NA[numeric])
NA2


In [ ]:

plt.figure(figsize=(20,11))
sns.boxplot(NA2)
plt.xticks(rotation=90)
plt.show()


In [ ]:
# hight Correlation Features

plt.figure(figsize=(20,11))
sns.boxplot(NA2[pair])
plt.xticks(rotation=90)
plt.show()


#### categoric data transformation

In [ ]:
from sklearn.preprocessing import LabelEncoder

NA3=z_scores
le = LabelEncoder()
NA3['Class'] = le.fit_transform(NA[categoric])
NA3


### 3) PCA : 

In [ ]:
from sklearn.decomposition import PCA

pca=PCA(n_components=3)
pca.fit(NA3)


In [ ]:
pca.components_, pca.explained_variance_, pca.explained_variance_ratio_


In [ ]:
X_pca=pca.fit_transform(NA3)
c=NA3['tcpEstabResets']#['ifInDiscards11'] # ['tcpEstabResets']     

plt.figure(figsize=(8,10))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=c, cmap='viridis')  # 'y' is the target variable for
#coloring points

plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title(f'PCA Scatter {c}')
plt.colorbar()  # Add a colorbar if you're using class labels for coloring
plt.show()


## <b><span style="color: #FF0000"> Interpretation:</span> </b>
There is 2 clusters in 2nd component arround 4 and 6. These clusters not depending on ['ifOutOctets11'] nor ['Class'].<br>
First Investigations show that these clusters are within NA3['ifInDiscards11']>2


### 4) Data Visualization:

### Heatmap

In [ ]:
plt.figure(figsize=(15,28))
#mask for upper triangle
mask=np.zeros_like(NA[numeric].corr(), dtype=bool)
mask[np.triu_indices_from(mask)]=True
#Generate custom diverging colormap
#cmap=sns.color_palette("flare")
cmap=sns.diverging_palette(h_neg=12, h_pos=12, s=100, l=40, sep=2, n=10, center='light', as_cmap=False)
#Heatmap with mask with correct aspect ratio
sns.heatmap(NA[numeric].corr(), mask=mask, cmap=cmap, vmin=-1,vmax=1,
            center=0, square=True, linewidths=0.5,annot=True, fmt='.2f', cbar_kws={'shrink':0.5})
plt.show()


In [ ]:
#PairPlot 

plt.figure(figsize=(20,12))
sns.pairplot(NA, hue='ifOutOctets11' ,palette='BuPu', kind='scatter', markers='^' ,dropna=True, height=3)
#palette='BuPu'
plt.legend()
plt.show()


### Heatmap with z scores: 

In [ ]:
plt.figure(figsize=(15,28))
#mask for upper triangle
mask=np.zeros_like(z_scores[numeric].corr(), dtype=bool)
mask[np.triu_indices_from(mask)]=True
#Generate custom diverging colormap
#cmap=sns.color_palette("flare")
cmap=sns.diverging_palette(h_neg=12, h_pos=12, s=100, l=40, sep=2, n=10, center='light', as_cmap=False)
#Heatmap with mask with correct aspect ratio
sns.heatmap(NA3[numeric].corr(), mask=mask, cmap=cmap, vmin=-1,vmax=1,
            center=0, square=True, linewidths=0.5,annot=True, fmt='.2f', cbar_kws={'shrink':0.5})
plt.show()


useless feature: tcp Estab reset, so: <br>
pair=['ifOutOctets11','ifInDiscards11', 'tcpOutRsts', 'tcpOutSegs','udpOutDatagrams',
      'ipOutRequests','tcpEstabResets', 'ipInAddErrors']

In [ ]:
#List with most relevant features according to correlation scores
pair=['ifOutOctets11','ifInDiscards11', 'tcpOutRsts', 'tcpOutSegs','udpOutDatagrams',
      'ipOutRequests','Class','ipInAddrErrors']
pair2=['ifOutOctets11','ifInDiscards11', 'tcpOutRsts','ipOutRequests', 'Class']


In [ ]:
#PairPlot with most relevant features according to correlation scores
plt.figure(figsize=(16,12))
sns.pairplot(NA3[pair], hue='Class' ,palette='bright', kind='scatter', markers='^' ,dropna=True, height=3)
#palette='BuPu'
plt.legend()
plt.show()


In [ ]:
#ItInDiscard & Tcpout need to be discarded because of their behaviour with all relevant features.
pair=['ifOutOctets11', 'tcpOutSegs','udpOutDatagrams','ipOutRequests','Class','ipInAddrErrors']

In [ ]:
from matplotlib import colormaps

list(colormaps)

In [ ]:

y='tcpOutSegs'
for x in pair:
    if x!=y:
        plt.figure(figsize=(8,3))
        plt.scatter(z_scores[x], z_scores[y], marker='o', edgecolors='b')
        plt.title(f'Relationship between {x} and {y}')
        plt.xlabel(x)
        plt.ylabel(y)
        plt.legend(loc='best')
        plt.show()


In [ ]:
NA3['ipInAddrErrors'].describe()


# II. Modeling

### 1) Data Splitting :

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test=train_test_split(NA3, test_size=0.3, random_state=404 )
X_train.shape, X_test.shape
X_train.describe()


### 2) Isolation Forest :

In [ ]:
from sklearn.ensemble import IsolationForest

Iso=IsolationForest(n_estimators=200,max_samples='auto', contamination=0.05,
                    max_features=1.0,random_state=404)
Iso.fit(X_train)


In [ ]:
anomaly=Iso.decision_function(X_test)
anomaly_predict=X_test[anomaly<0]
anomaly_predict.describe()
#anomaly.size

In [ ]:
anomalie=Iso.decision_function(X_train)
#y_true=
Anom=[]
for a in X_train['Class']:
    if a==6 or a==7: Anom.append(1)
    else: Anom.append(0)
XX= X_train.iloc[:,0:34]     
XX['Class']=Anom
XX

In [ ]:
from sklearn.metrics import roc_auc_score

y_true=XX['Class']
anomalie=Iso.decision_function(XX)
auc = roc_auc_score(y_true ,-anomalie)  
print(f'{auc:.4f}')


this score shows that the model predicted better than random guessing

In [ ]:
from sklearn.metrics import roc_curve

# Compute the false positive rate (FPR), true positive rate (TPR), and thresholds
fpr, tpr, thresholds = roc_curve(y_true, anomalie)
# Plot the ROC curve
plt.plot(fpr, tpr, label='ROC Curve (AUC = %0.3f)' % auc, color='r')
plt.plot([0, 1], [0, 1], 'k--')  # Plotting the diagonal line for reference
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
lab=pd.DataFrame(le.inverse_transform(X_train['Class']),columns=['Class'])
Counter(lab['Class'])

### 3) Gaussian Distribution :

* Recall that the Gaussian distribution is given by

   $$ p(x ; \mu,\sigma ^2) = \frac{1}{\sqrt{2 \pi \sigma ^2}}\exp^{ - \frac{(x - \mu)^2}{2 \sigma ^2} }$$


In [ ]:
from scipy.stats import norm

data=pd.concat([X_train['tcpOutSegs'],X_train['ipOutRequests']],axis=1, join='outer', 
               ignore_index=False,names=['ipOutRequests'])
#data=X_train.iloc[:, 0:-1] 
# Estimate the mean and standard deviation from the data
mu, sigm = np.mean(data), np.std(data)

# Generate a range of values for the x-axis
x = np.linspace(mu - 3*sigm, mu + 3*sigm, 100)

# Calculate the probability density function (PDF) using the estimated parameters
pdf = norm.pdf(x, mu, sigm)
plt.figure(figsize=(12,8))
# Plot the histogram of the data
plt.hist(data, bins=50, density=True, alpha=0.5, label='Data')
# Plot the Gaussian distribution
plt.plot(x, pdf, 'r-', label='Gaussian Distribution')
# Set the plot labels and legend
plt.xlabel('Values')
plt.ylabel('Probability Density')
plt.title('Gaussian Distribution Fit')
plt.legend(loc='best')

# Show the plot
plt.show()
print(f'mu={mu} +/-{sigm} ')


In [ ]:
from matplotlib.gridspec import GridSpec

v_features = X_train.iloc[:, 0:-1].columns
plt.figure(figsize=(22, 12*4))
num_rows=len(X_train.columns)-1
gs = GridSpec(num_rows, 4)

for i, cn in enumerate(v_features):
    ax = plt.subplot(gs[i])
    sns.histplot(X_train.loc[(X_train['Class'] >= 6), cn], bins=50, kde=True, color='red', label='Anomaly')
    sns.histplot(X_train.loc[X_train['Class'] <6, cn], bins=50, kde=True, color='blue', label='Normal')
    ax.set_xlabel('')
    ax.set_title('Feature: ' + str(cn))
    ax.legend()

plt.tight_layout()
plt.show()


list of features that show anomaly distribution in our gridspec: <br>

In [ ]:
Ano=['ifOutOctets11', 'ifInUcastPkts11', 'ifInUcastPkts11', 'ifInNUcastPkts11',
     'ifOutUcastPkts11','tcpPassiveOpens', 'ipInDiscards',
     'icmpOutMsgs', 'icmpInDestUnreachs', 'icmpOutEchoReps', 'Class']
X_train[Ano]


In [ ]:
from scipy.stats import norm

out=X_train[X_train['Class']>=6]
outliers=out[Ano].iloc[:,0:-1]

data=X_train[Ano].iloc[:, 0:-1]
mu, sigm = np.mean(data), np.std(data)
# Generate a range of values for the x-axis
x = np.linspace(mu - 3*sigm, mu + 3*sigm, 100)
# Calculate the probability density function (PDF) using the estimated parameters
pdf = norm.pdf(x, mu, sigm)
plt.figure(figsize=(12,8))
# Plot the histogram of the data
plt.hist(data, bins=50, density=True, alpha=0.5, label='Data')
# Plot the Gaussian distribution
plt.plot(x, pdf, 'r-', label='Gaussian Distribution')
# Set the plot labels and legend
plt.xlabel('Values')
plt.ylabel('Probability Density')
plt.title('Gaussian Distribution Fit')
plt.legend(loc='best')

# Show the plot
plt.show()
print(f'mu={mu} +/-{sigm} ')


#### a\ Classify Feature importance: 

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score

def print_score(clf, X_train, X_test, Y_train, Y_test, train=True):
    "Print the accuracy score, Classification report and confusion matrix"
    if train:
        "Train performance"
        Y_train_pred=clf.predict(X_train)
        print(f'Train Results:\n')
        print(f'MSE Train={mean_squared_error(Y_train,Y_train_pred):.4f}')
        print(f'R² Train={r2_score(Y_train,Y_train_pred):.4f}')
        
   
    elif train==False:
        "Test performance"
        Y_test_pred=clf.predict(X_test)
        print(f'Test Results:\n')
        print(f'MSE Test ={mean_squared_error(Y_test,Y_test_pred):.4f}')
        print(f'R² Test ={r2_score(Y_test,Y_test_pred):.4f}')


In [ ]:
X=X_train.iloc[:, 0:-1]
Y=X_train.iloc[:, -1]
X1_test=X_test.iloc[:, 0:-1]
Y1_test=X_test.iloc[:, -1]


In [ ]:
from sklearn.ensemble import RandomForestRegressor

#Criterion: 'poisson', 'squared_error', 'friedman_mse', 'absolute_error'
forest=RandomForestRegressor(n_estimators=500, criterion='squared_error', random_state=422, n_jobs=-1)
forest.fit(X,Y)
forest


In [ ]:
print_score(forest, X, X1_test,Y, Y1_test, train=True)
print('------------------------------------------>>')
print_score(forest, X, X1_test,Y, Y1_test, train=False)


In [ ]:
import plotly
import plotly.graph_objs as go
from plotly.offline import *

x, y = (list(x) for x in zip(*sorted(zip(forest.feature_importances_, 
                                         X.columns),reverse = False)))
trace2 = go.Bar(x=x ,y=y,marker=dict(
    color=x,colorscale = 'Viridis',reversescale = True),
                name='Random Forest Feature importance',orientation='h',)

layout = dict(title='Barplot of Feature importances',width = 600, height = 1000,
              yaxis=dict(showgrid=False,showline=False,showticklabels=True,),
              margin=dict(l=300,),)
# domain=[0, 0.85],
fig1 = go.Figure(data=[trace2], layout=layout)
iplot(fig1)


In [ ]:

Features_I=[]
for importance,name in sorted(zip(forest.feature_importances_,X.columns),reverse = True):
    if importance > 0.03 :
        Features_I.append(name)
Features_I


Need to separate dataset : normal and anomaly, for calculating cross validation

In [ ]:
NA_N=NA3[NA3['Class']<6 ]
NA_A=NA3[NA3['Class']>=6 ]
NA_N


In [ ]:
from sklearn.model_selection import train_test_split

X_trainN,X_testN=train_test_split(NA_N, test_size=0.3, random_state=65 )
X_trainA,X_testA=train_test_split(NA_A, test_size=0.3, random_state=40 )
X_trainA.shape, X_testA.shape
X_trainA.describe()


In [ ]:

xtrain_strip_v0 = X_trainN[Features_I]
xtrain_strip_v1 = X_trainA[Features_I]
ytrain_v0=X_trainN['Class']
ytrain_v1=X_trainA['Class']
xtest_v0=X_testN[Features_I]
xtest_v1=X_testA[Features_I]
ytest_v0=X_testN['Class']
ytest_v1=X_testA['Class']
xtrain_strip_v0, ytrain_v0
#xtest_v1, ytest_v1


In [ ]:
from sklearn.ensemble import RandomForestRegressor

#Criterion: 'poisson', 'squared_error', 'friedman_mse', 'absolute_error'
forest=RandomForestRegressor(n_estimators=100, criterion='squared_error', random_state=82, n_jobs=-1)
forest.fit(xtrain_strip_v0,ytrain_v0)
forest


In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(forest, xtrain_strip_v0, ytrain_v0, cv=5, n_jobs=-1,
                         scoring='r2') # for regression tasks: "neg_mean_squared_error",neg_mean_absolute_error","r2"
scoresA = cross_val_score(forest, xtrain_strip_v1, ytrain_v1, cv=5, n_jobs=-1,
                         scoring='r2')

print(f'Scores Normal: {scores}\n')
print(f'Scores Anomaly: {scoresA}')


#### b\ Select threshold for best F1 score :

* Then, you can compute precision and recall by: 
   $$\begin{aligned}
   prec&=&\frac{tp}{tp+fp}\\
   rec&=&\frac{tp}{tp+fn},
   \end{aligned}$$ where
    * $tp$ is the number of true positives: the ground truth label says it’s an anomaly and our algorithm correctly classified it as an anomaly.
    * $fp$ is the number of false positives: the ground truth label says it’s not an anomaly, but our algorithm incorrectly classified it as an anomaly.
    * $fn$ is the number of false negatives: the ground truth label says it’s an anomaly, but our algorithm incorrectly classified it as not being anomalous.

  * The $F_1$ score is computed using precision ($prec$) and recall ($rec$) as follows:
    $$F_1 = \frac{2\cdot prec \cdot rec}{prec + rec}$$ 


In [ ]:
from scipy.stats import multivariate_normal

def estimateGaussian(dataset):
    mu = np.mean(dataset, axis=0)
    sigma = np.std(dataset)# rowvar=False)
    return mu, sigma

def multivariateGaussian(dataset,mu,sigma):
    p = multivariate_normal(mean=mu, cov=sigma)
    return p.pdf(dataset)


In [ ]:
mu, sigma=estimateGaussian(xtrain_strip_v0)
mu


In [ ]:
mu, sigma=estimateGaussian(xtrain_strip_v0)
mulg_v0=multivariateGaussian(xtrain_strip_v0,mu,sigma)


In [ ]:
mult_v1=multivariateGaussian(xtest_v1,mu,sigma)
mulg_v1=multivariateGaussian(xtrain_strip_v1,mu,sigma)
mulg_v1


In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

def selectThresholdByCV(probs,gt):
    best_epsilon = 0
    best_f1 = 0
    f = 0
    farray = []
    Recallarray = []
    Precisionarray = []
    epsilons = (-0.001,0, 1e-3, 5e-3, 1e-2,5e-2, 1) #(0.0000e+00, 1.0527717316e-70, 1.0527717316e-50, 1.0527717316e-24)
    #epsilons = np.asarray(epsilons)
    for epsilon in epsilons:
        predictions = (mulg_v0 < epsilon)
        f = f1_score(ytrain_v0, predictions, average = "weighted")
        Recall = recall_score(ytrain_v0, predictions, average = "weighted")
        Precision = precision_score(ytrain_v0, predictions, average = "weighted",zero_division=1)
        farray.append(f)
        Recallarray.append(Recall)
        Precisionarray.append(Precision)
        print ('For below Epsilon')
        print(epsilon)
        print ('F1 score , Recall and Precision are as below')
        print ('Best F1 Score %f' %f)
        print ('Best Recall Score %f' %Recall)
        print ('Best Precision Score %f' %Precision)
        print ('-'*40)
        if f > best_f1:
            best_f1 = f
            best_recall = Recall
            best_precision = Precision
            best_epsilon = epsilon    
    fig = plt.figure()
    ax = fig.add_axes([0.1, 0.5, 0.7, 0.3])
    #plt.subplot(3,1,1)
    plt.plot(farray ,"ro")
    plt.plot(farray)
    ax.set_xticks(range(7))
    ax.set_xticklabels(epsilons,rotation = 60 ,fontsize = 'medium' )
    ax.set_ylim((0,1.0))
    ax.set_title('F1 score vs Epsilon value')
    ax.annotate('Best F1 Score', xy=(best_epsilon,best_f1), xytext=(best_epsilon,best_f1))
    plt.xlabel("Epsilon value") 
    plt.ylabel("F1 Score") 
    plt.ylim(0, 0.2)
    plt.show()
    fig = plt.figure()
    ax = fig.add_axes([0.1, 0.5, 0.9, 0.3])
    #plt.subplot(3,1,2)
    plt.plot(Recallarray ,"ro")
    plt.plot(Recallarray)
    ax.set_xticks(range(7))
    ax.set_xticklabels(epsilons,rotation = 60 ,fontsize = 'medium' )
    ax.set_ylim((0,1.0))
    ax.set_title('Recall vs Epsilon value')
    ax.annotate('Best Recall Score', xy=(best_epsilon,best_recall), xytext=(best_epsilon,best_recall))
    plt.xlabel("Epsilon value") 
    plt.ylabel("Recall Score") 
    plt.ylim(0, 0.2)
    plt.show()
    fig = plt.figure()
    ax = fig.add_axes([0.1, 0.5, 0.9, 0.3])
    #plt.subplot(3,1,3)
    plt.plot(Precisionarray ,"ro")
    plt.plot(Precisionarray)
    ax.set_xticks(range(7))
    ax.set_xticklabels(epsilons,rotation = 60 ,fontsize = 'medium' )
    ax.set_ylim((0,1.0))
    ax.set_title('Precision vs Epsilon value')
    ax.annotate('Best Precision Score', xy=(best_epsilon,best_precision), xytext=(best_epsilon,best_precision))
    plt.xlabel("Epsilon value") 
    plt.ylabel("Precision Score") 
    plt.ylim(0.6, 1)
    plt.show()
    return best_f1, best_epsilon


In [ ]:
fscore, ep= selectThresholdByCV(mulg_v0,ytest_v1)


In [ ]:
ep

Epsilon value = 1e-2 is selected as threshold to identify Anomalous values 
now time to Predict and calculate  F1 , Recall and Precision score for our Test Dataset

#### remake full X_train and X_test:

In [ ]:
train_X = pd.concat([X_trainN[Features_I],X_trainA[Features_I] ],axis=0)
test_X = pd.concat([X_testN[Features_I],X_testA[Features_I]],axis=0)
test_Y=pd.concat([X_testN['Class'],X_testA['Class']],axis=0)
test_X


#### c\ Making predictions :

In [ ]:
p_test=multivariateGaussian(test_X,mu,sigma)
p_test


In [ ]:
predictions = (p_test < ep)
Recall = recall_score(test_Y, predictions, average = "weighted")    
Precision = precision_score(test_Y, predictions, average = "weighted",zero_division=1)
F1score = f1_score(test_Y, predictions, average = "weighted")    
print ('F1 score , Recall and Precision for Test dataset')
print ('Best F1 Score %.4f' %F1score)
print ('Best Recall Score %.4f' %Recall)
print ('Best Precision Score %.4f' %Precision)


##### Lets Visualize our predictions in below scatter plot 


In [ ]:
Features_I

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))
ax.scatter(test_X['tcpEstabResets'],test_X['ifOutNUcastPkts11'],marker="o", color="lightBlue")
ax.set_title('Anomalies(in red) vs Predicted Anomalies(in Green)')
for i, txt in enumerate(test_X.index):
       if test_Y[txt] == 1 :
            ax.annotate('*', (test_X['tcpEstabResets'][txt],test_X['ifOutNUcastPkts11'][txt]),fontsize=24,color='Red')
       if predictions[i] == True :
            ax.annotate('o', (test_X['tcpEstabResets'][txt],test_X['ifOutNUcastPkts11'][txt]),fontsize=15,color='Green')
plt.legend(loc='best')
plt.show()



Precision of around 90% with Recall of 10%, 


In [ ]:
p_test=multivariateGaussian(xtrain_strip_v0,mu,sigma)
np.info(object=p_test)
xtrain_strip_v0.info()


In [ ]:

xtrain_strip_v0 = X_trainN[Features_I]
xtrain_strip_v1 = X_trainA[Features_I]
ytrain_v0=X_trainN['Class']
ytrain_v1=X_trainA['Class']
xtest_v0=X_testN[Features_I]
xtest_v1=X_testA[Features_I]
ytest_v0=X_testN['Class']
ytest_v1=X_testA['Class']
#xtrain_strip_v0, ytrain_v0
#xtest_v1, ytest_v1


In [ ]:
predictions = (p_test < ep)
Recall = recall_score(ytrain_v0, predictions, average = "weighted")    
Precision = precision_score(ytrain_v0, predictions, average = "weighted",zero_division=1)
F1score = f1_score(ytrain_v0, predictions, average = "weighted")    
print ('F1 score , Recall and Precision for Test dataset')
print ('Best F1 Score %.4f' %F1score)
print ('Best Recall Score %.4f' %Recall)
print ('Best Precision Score %.4f' %Precision)


Precision=85%, with a recall (anmalies predicted)=18%